In [ ]:
import copy, sys, pandas as pd
import random
import time
import numpy as np
import subprocess 

sys.path.append("../../")
sys.path.append("../../inferring")
sys.path.append("../../utils/DFA")
from importlib import reload


import inferring.Inferring as Inferring, inferring.InferringDFA as InferringDFA
import utils.automats.DFA
import utils.advice_systems.SRS as SRS

reload(SRS)
reload(Inferring)
reload(InferringDFA)
reload(utils.automats.DFA)

from inferring.Inferring import Inferring
from utils.automats.DFA.DFA import DFA
from utils.advice_systems.SRS import SRS

from inferring.InferringDFA import InferringDFA

In [2]:
def run_learning_process(target, advice_system=None, check_consistency=False, equiv_query_fashion="BFS", debug=False):
    _dfa = copy.deepcopy(target)
    learn_dfa = InferringDFA(_dfa, 
                                advice_system, 
                                check_consistency=check_consistency, 
                                equiv_query_fashion=equiv_query_fashion,
                                debug=debug)
        
    dfa, cnt, cnt_ex = learn_dfa.run(counterexamples=True)
    return copy.deepcopy(dfa), cnt, len(cnt_ex) +1 

In [16]:
class Record:

    def __init__(self,
                 d1,
                 d2,
                 conv,
                 lstar_with_advice,
                 target=None,
                 ttt_eq=None, 
                 ttt_wa_eq=None):
        self.d1 = d1
        self.d2 = d2
        self.conv = conv
        self.lstar_with_advice = lstar_with_advice
        self.ttt_eq = ttt_eq
        self.ttt_wa_eq = ttt_wa_eq 
        self.target = target

    def print_record(self):
        print(
            f"|d1| = {self.d1}, |d2| = {self.d2}, |conv| = {self.conv}, (mq, eq) = {self.lstar_with_advice}"
        )
    def get_complete_description(self):
        return self.target.print_complete_description()
    
    def print_target(self):
        print(self.target.print_complete_description())

In [28]:
def getNumberofEQ(s):
    for l in s.splitlines():
        if l.startswith("Learning rounds"):
            return l.split()[-1]

In [49]:
"""
    Files TTTExample1.java and TTTExample1_withAS.java should be placed in the following directory: examples/src/main/java/de/learnlib/example
    Before executing TTTExample1.java file build learlib project:                                   mvn clean install 
"""


number_of_itreations = 65

#  Fixed random seeds for reproducibility
seeds = [i for i in range(number_of_itreations)]

# The alphabet of the DFA
input_signs = ["a", "b", "c"]

# The bound on the number of states
max_number_of_states = 40

# The list to aggregate the results
results = []

_ = subprocess.run(["cd ../../../learnlib/examples ; mvn clean install; cd ../../magisterka/test_algorithm/TTT"], shell=True, capture_output=True, text=True)
i = 0
while i < number_of_itreations:
    random.seed(seeds[i])
    i += 1
    print(f"iter nr: {i}")

    # Create two random DFAs
    dfa1 = DFA()
    dfa2 = DFA()
    dfa1.create_random_dfa(
        Q=random.randint(max_number_of_states // 2, max_number_of_states),
        input_signs=input_signs,
    )
    dfa2.create_random_dfa(
        Q=random.randint(max_number_of_states // 2, max_number_of_states),
        input_signs=input_signs,
    )

    # Run learning of the language of DFA to minimize dfa1 and dfa2
    d1, _, _ = run_learning_process(target=copy.deepcopy(dfa1))
    d2, _, _ = run_learning_process(target=copy.deepcopy(dfa2))

    if d1.Q < 2 or d2.Q < 2:
        continue
        # Create two random DFAs
        dfa1 = DFA()
        dfa2 = DFA()
        dfa1.create_random_dfa(
            Q=random.randint(max_number_of_states // 2, max_number_of_states),
            input_signs=input_signs,
        )
        dfa2.create_random_dfa(
            Q=random.randint(max_number_of_states // 2, max_number_of_states),
            input_signs=input_signs,
        )

        # Run learning of the language of DFA to minimize dfa1 and dfa2
        d1, _, _ = run_learning_process(target=copy.deepcopy(dfa1))
        d2, _, _ = run_learning_process(target=copy.deepcopy(dfa2))

    # Create a convolution DFA based on d1 and d2
    conv_dfa = DFA()
    conv_dfa.create_convolution(dfa1=d1, dfa2=d2)

    # Learn conv_dfa with advice, and store d - learned automaton 
    start_t = time.time()
    d, lstar_with_advice, lstar_ex_with_advice = run_learning_process(
        target=conv_dfa, advice_system=SRS(), check_consistency=True
    )
    end_t = time.time()

    print("ex time with AS: ", end_t - start_t)
    print(f"|d1.Q| = {d1.Q}, |d2.q| = {d2.Q}, |d.Q| = {d.Q}")
    d.type = DFA.CONV_DFA

    results.append(
        Record(
            d1=d1.Q,
            d2=d2.Q,
            conv=d.Q,
            lstar_with_advice=(lstar_with_advice, lstar_ex_with_advice),
            target=copy.deepcopy(d),
        )
    )
    results[-1].print_record()
    f = open("../../../learnlib/examples/src/main/java/de/learnlib/example/DfaEx" + ".txt", "w")
    f.write(results[-1].get_complete_description())
    f.close()

    ttt_output = subprocess.run(["cd ../../../learnlib/examples ; mvn exec:java -q -Dexec.mainClass=\"de.learnlib.example.TTTExample1\" ; cd ../../magisterka/test_algorithm/TTT"], shell=True, capture_output=True, text=True)
    print("TTT done")
    ttt_wa_output = subprocess.run(["cd ../../../learnlib/examples ; mvn exec:java -q -Dexec.mainClass=\"de.learnlib.example.TTTExample1_withAS\" ; cd ../../magisterka/test_algorithm/TTT"], shell=True, capture_output=True, text=True)
    print("TTT with AS done")

    results[-1].ttt_eq = int(getNumberofEQ(ttt_output.stdout))
    results[-1].ttt_wa_eq = int(getNumberofEQ(ttt_wa_output.stdout))
    

iter nr: 1
ex time with AS:  60.76960778236389
|d1.Q| = 30, |d2.q| = 33, |d.Q| = 990
|d1| = 30, |d2| = 33, |conv| = 990, (mq, eq) = ([519133, 2], 160)
TTT done
TTT with AS done
iter nr: 2
ex time with AS:  12.416016340255737
|d1.Q| = 22, |d2.q| = 23, |d.Q| = 506
|d1| = 22, |d2| = 23, |conv| = 506, (mq, eq) = ([160267, 2], 92)
TTT done
TTT with AS done
iter nr: 3
iter nr: 4
ex time with AS:  68.86998796463013
|d1.Q| = 26, |d2.q| = 37, |d.Q| = 962
|d1| = 26, |d2| = 37, |conv| = 962, (mq, eq) = ([517356, 3], 165)
TTT done
TTT with AS done
iter nr: 5
ex time with AS:  22.589881896972656
|d1.Q| = 27, |d2.q| = 25, |d.Q| = 675
|d1| = 27, |d2| = 25, |conv| = 675, (mq, eq) = ([237219, 2], 106)
TTT done
TTT with AS done
iter nr: 6
ex time with AS:  244.95925116539001
|d1.Q| = 37, |d2.q| = 38, |d.Q| = 1406
|d1| = 37, |d2| = 38, |conv| = 1406, (mq, eq) = ([1215106, 2], 262)
TTT done
TTT with AS done
iter nr: 7
ex time with AS:  114.02470302581787
|d1.Q| = 37, |d2.q| = 30, |d.Q| = 1110
|d1| = 37, |

KeyboardInterrupt: 

In [50]:
results.sort(key=lambda x: x.conv)
# Remove degenerated cases (Conv DFA of size 1)
results = [r for r in results if r.conv != 1]

print(f"number of tests: {len(results)}")
# for i, r in enumerate(results):
#     r.print_record()

number of tests: 51


In [51]:
def create_table(columns, data):
    columns = pd.MultiIndex.from_tuples(columns)
    df = pd.DataFrame(data, columns=columns)

    avg_eq_red = df[(('Reduction', 'EQ'))].mean()
    max_eq_red = df[(('Reduction', 'EQ'))].max()
    min_eq_red = df[(('Reduction', 'EQ'))].min()
    empty_row = pd.DataFrame([[""] * df.shape[1]], columns=df.columns)

    def set_params(row, params):
        for k, v in params:
            row[columns[k]] = v 

    max_red_row = copy.deepcopy(empty_row)
    set_params(row=max_red_row, params=[(0, "Max reduction"), (-1, int(max_eq_red))])
    min_red_row = copy.deepcopy(empty_row)
    set_params(row=min_red_row, params=[(0, "Min reduction"), (-1, int(min_eq_red))])
    avg_red_row = copy.deepcopy(empty_row)
    set_params(row=avg_red_row, params=[(0, "Average reduction"), (-1, int(avg_eq_red))])

    rows = [empty_row, max_red_row, min_red_row, avg_red_row]
    for row in rows:
        df = pd.concat([df, row], ignore_index=True)

    df[('Reduction', 'EQ')] = df[('Reduction', 'EQ')].apply(lambda x: f'{x}%' if isinstance(x, int) else x)
    return df

In [52]:
ttt_eqs = [r.ttt_eq for r in results]
ttt_wa_eqs = [r.ttt_wa_eq for r in results]
red = ((np.array(ttt_eqs) - np.array(ttt_wa_eqs)) / np.array(ttt_eqs)) * 100

columns = [('Target language', 'conv(DFA1, DFA2)'), ('Target language', 'DFA1'), ('Target language', 'DFA2'), ('TTT', 'EQ'), ('TTT with advice', 'EQ'), ('Reduction', 'EQ')]
data = {
    columns[0]: [r.conv for r in results],
    columns[1]: [r.d1 for r in results],
    columns[2]: [r.d2 for r in results],
    columns[3]: ttt_eqs,
    columns[4]: ttt_wa_eqs,
    columns[5]: [int(r) for r in red]
}

df = create_table(columns=columns, data=data)
display(df)

Target language             TTT TTT with advice Reduction
     conv(DFA1, DFA2) DFA1 DFA2    EQ              EQ        EQ
0                 462   22   21   451              17       96%
1                 475   19   25   464              17       96%
2                 480   24   20   465              16       96%
3                 506   22   23   490              10       97%
4                 506   23   22   493               6       98%
5                 513   19   27   508              15       97%
6                 522   29   18   517              12       97%
7                 551   19   29   535              19       96%
8                 558   18   31   543              10       98%
9                 572   22   26   548               6       98%
10                621   23   27   613               7       98%
11                625   25   25   605               9       98%
12                627   33   19   602               8       98%
13                638   29   22   629               9       98%
14                650   25   26   639              18       97%
15                675   27   25   662               8       98%
16                682   22   31   669               4       99%
17                720   36   20   710               9       98%
18                720   36   20   712               9       98%
19                736   32   23   725              13       98%
20                780   26   30   759              23       96%
21                783   27   29   758               8       98%
22                792   36   22   769              17       97%
23                819   21   39   796              24       96%
24                836   38   22   816               6       99%
25                837   27   31   820              12       98%
26                840   30   28   816              10       98%
27                841   29   29   820               5       99%
28                850   25   34   806               8       99%
29                858   33   26   823               2       99%
30                875   25   35   849               6       99%
31                875   25   35   865              19       97%
32                875   25   35   853              18       97%
33                888   24   37   870               3       99%
34                888   24   37   871              23       97%
35                912   24   38   892               5       99%
36                925   37   25   888               7       99%
37                962   26   37   937               4       99%
38                990   30   33   974              12       98%
39                992   32   31   984               9       99%
40               1050   30   35  1011               8       99%
41               1073   29   37  1040              21       97%
42               1110   37   30  1103              13       98%
43               1221   37   33  1206              23       98%
44               1224   34   36  1212              16       98%
45               1260   36   35  1187              13       98%
46               1295   35   37  1252              14       98%
47               1330   35   38  1292               9       99%
48               1332   37   36  1270              12       99%
49               1404   39   36  1391              16       98%
50               1406   37   38  1327               3       99%
51                                                             
52      Max reduction                                       99%
53      Min reduction                                       96%
54  Average reduction                                       97%